# Cleaning and analysis of the given data

#### Imports

In [110]:
import re
import json

#### Variables

In [111]:
json_list = []
DATA_NAME = 'stat-2019-06.txt'

#### functions

In [112]:
def check_bracket(json_string):
    counter, counter2, counter3 = 0, 0, 0
    counter += json_string.count("{")
    counter -= json_string.count("}")
    counter2 += json_string.count("[")
    counter2 -= json_string.count("]")
    counter3 += json_string.count("(")
    counter3 -= json_string.count(")")
    if counter == 0 and counter2==0 and counter3 == 0:
        return True
    print("SyntaxError: Bracket")
    return False

def check_syntax(json_string):
    try:
        json_str = json.loads(json_string)
        return True
    except:
        print("SyntaxError")
        return False

def check_elements(json_string):
    if json_string.find("\", \"sessionId\":\"")>=0:
        if json_string.find("\", \"robotName\":\"") >=0:
            if json_string.find("\", \"logLevel\":\"")>=0:
                if json_string.find("\", \"message\":")>=0:
                    return True
    print("Error: Element is missing or broken")
    return False

def validate_line(line):
    data_group = re.search("({.*})",line)
    try:
        json_string = data_group.group()
    except:
        return False, "o"
    if data_group and check_bracket(json_string) and check_syntax(json_string) and check_elements(json_string):
        json_dict = json.loads(data_group.group())
        return True, json_dict
    return False, "o"

## Reading of every usefull piece of data

In [113]:
with open(DATA_NAME,"rt") as data_file: 
    for line in data_file:
        if validate_line(line)[0]:
            json_list.append(validate_line(line)[1])


## Analysis 

#### Variables

In [114]:
list_actions = []
simulations = 0
robot_count = []
robot_names = []

In [116]:
for element in json_list:
    if type(element["message"]) is dict and element["message"]["action"]:
        if element["message"]["action"] not in list_actions:
            list_actions.append(element["message"]["action"])
            
for element in json_list:
    if element["robotName"] not in robot_names:
        robot_names.append(element["robotName"])
        robot_count.append([])
    if type(element["message"]) is dict and element["message"]["action"]:
        index = robot_names.index(element["robotName"])
        
        for action in list_actions:
            if element["message"]["action"] == action:
                robot_count[index].append(action)
        
for i, robot in enumerate(robot_names):
    print("RobotName:", robot)
    print()
    for action in list_actions:
        print(action ,":",robot_count[i].count(action))
    print()
    print()



RobotName: ev3lejosv1

Initialization : 101804
ProgramRunBack : 206
ProgramSource : 3330
UserLogin : 3990
ProgramRun : 15010
SimulationRun : 281234
UserLogout : 874
GalleryView : 2412
ProgramSave : 5618
ProgramImport : 3288
ProgramLoad : 4846
SimulationBackgroundUploaded : 2908
ProgramShare : 56
ProgramDelete : 86
GalleryLike : 6
ProgramCompileNative : 0
ProgramShareDelete : 8
GalleryShare : 0


RobotName: calliope2017

Initialization : 13424
ProgramRunBack : 246378
ProgramSource : 9416
UserLogin : 17590
ProgramRun : 0
SimulationRun : 299792
UserLogout : 4908
GalleryView : 12138
ProgramSave : 28552
ProgramImport : 6806
ProgramLoad : 33300
SimulationBackgroundUploaded : 0
ProgramShare : 114
ProgramDelete : 840
GalleryLike : 222
ProgramCompileNative : 0
ProgramShareDelete : 10
GalleryShare : 14


RobotName: sensebox

Initialization : 116
ProgramRunBack : 90
ProgramSource : 106
UserLogin : 10
ProgramRun : 0
SimulationRun : 0
UserLogout : 0
GalleryView : 46
ProgramSave : 2
ProgramImport : 